In [1]:
import cv2  # type: ignore
import numpy as np # type: ignore

In [2]:
def calculateError(setpoint: np.ndarray[int, int], current=np.array([0, 0])) -> float:
    # error = (setpoint - current) ** 2
    # return sum(error) ** 0.5
    return setpoint - current

In [3]:
def calculateProportionality(error, proportionality=1) -> float:
    return proportionality * error

In [4]:
def calculateIntegral(error, currentIntegralError, frameRate, proportionality=1) -> tuple[float, float]:
    integral = currentIntegralError + error * (1 / frameRate)
    return (integral, proportionality * integral)

In [5]:
def calculateDifferential(error, prevError, frameRate, proportionality=1) -> float:
    time = 1 / frameRate
    differential = (error - prevError) / time
    return differential * proportionality

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

success, img = cap.read()
cv2.imshow('Webcam', img)
current = np.array([0, 0], dtype=np.int64)
setpoint = np.array([100, 100], dtype=np.int64)
preverror = 0
interror = 0
cv2.circle(img, current, radius=10, color=(0, 0, 255), thickness=-1)

while 1:
    success, img = cap.read()
    cv2.imshow('Webcam', img)
    print((int(current[0]), int(current[1])))
    # cv2.circle(img, center=(int(current[0]), int(current[1])), radius=10, color=(0, 0, 255), thickness=-1)
    cv2.circle(img, center=(200, 200), radius=10, color=(0, 0, 255), thickness=-1)

    error = calculateError(current=current, setpoint=setpoint)

    properror = calculateProportionality(error, 1e-3)
    interror, currentInt = calculateIntegral(error, interror, 30, 1e-11)
    differror = calculateDifferential(error, preverror, 30, 2 * 1e-2)
    print(f"Error: {error}, prop: {properror}, int: {interror}, diff: {differror}")
    preverror = error

    current += np.int64(properror + interror + differror)
    # current[1] += int(properror + interror + differror)

    if cv2.waitKey(0) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


(0, 0)
Error: [100 100], prop: [0.1 0.1], int: [3.33333333 3.33333333], diff: [300. 300.]
(303, 303)
Error: [-203 -203], prop: [-0.203 -0.203], int: [-3.43333333 -3.43333333], diff: [-909. -909.]
(-609, -609)
Error: [709 709], prop: [0.709 0.709], int: [20.2 20.2], diff: [2736. 2736.]
(2147, 2147)
Error: [-2047 -2047], prop: [-2.047 -2.047], int: [-48.03333333 -48.03333333], diff: [-8268. -8268.]
(-6171, -6171)
Error: [6271 6271], prop: [6.271 6.271], int: [161. 161.], diff: [24954. 24954.]


In [7]:
# Open the default camera (0)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    # Read frame from the camera
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Could not read frame.")
        break

    # Display the frame
    cv2.imshow("Camera Feed", frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()